In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
import os

os.getcwd()


'c:\\Users\\nik\\Desktop\\Berkeley_Projects\\LoadLanguageModel\\bin'

In [86]:
df = pd.read_csv('../data\MFRED_Aggregates_15min_2019Q1-Q4.csv', index_col=0, parse_dates=True).filter(regex=r'^.*kW$')
df = df.iloc[:, 1:]
scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

df = df.resample('1H').mean()
df = df.stack().reset_index().rename(columns={'level_0': 'timestamp', 'level_1': 'apartment', 0: 'kW'})

# drop the _Kw from the apartment names

df['apartment'] = df['apartment'].str.replace('_kW', '')



In [87]:
# reset index and call the first column apartment
df_meta = pd.read_csv("../data/meta_data.csv", index_col=0).iloc[:-1, :]
df_meta = df_meta.reset_index().rename(columns={'AP ': 'apartment'})
df_meta.columns

Index(['Apt. group', 'Time-averaged real power (σ) [W]',
       'Number of bedrooms (σ)', 'Number of all rooms (σ)',
       'Apt. area (σ) [m2]'],
      dtype='object')

In [88]:


df_meta.rename(columns={'Apt. group': 'apartment'}, inplace=True)


In [89]:
df_meta

,apartment,Time-averaged real power (σ) [W],Number of bedrooms (σ),Number of all rooms (σ),Apt. area (σ) [m2]
0,AG01,42,1.2,3.2,62
1,AG02,81,1.3,3.9,77
2,AG03,105,1.3,3.3,65
3,AG04,119,1.1,3.1,58
4,AG05,141,1.5,4.2,91
5,AG06,156,1.3,3.8,77
6,AG07,166,1.4,4.2,85
7,AG08,179,1.3,3.8,77
8,AG09,189,1.4,3.7,86
9,AG10,203,1.3,3.7,81


In [90]:

# make columns in df with meta data information

df = df.merge(df_meta, on='apartment', how='left')


In [95]:
# sort by apartment and timestamp

df = df.sort_values(['apartment', 'DateTimeUTC'])

In [97]:
df.to_csv('../data/apartment_data_prepared.csv', index=False)